In [10]:
# %pip install essentia
# %pip install librosa
# %pip install yt_dlp

In [73]:
import librosa
import essentia.standard as es
import numpy as np
import pandas as pd
import os

In [70]:
""" TEMPO (BPM) """
def get_tempo(y, sr):
    # Estimar el tempo
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    # El resultado 'tempo' es un número flotante, p. ej., 129.199
    print(tempo)
    return tempo

""" KEY (Tonalidad) """
def get_key(audio):
    key_extractor = es.KeyExtractor()
    key, scale, strength = key_extractor(audio)
    
    return(key, scale) # 'key' será la tónica (p. ej., "C") y 'scale' la modalidad (p. ej., "major")

""" LOUDNESS (Volumen) """

def get_loudness(audio):
    loudness = es.Loudness()(audio)
    return loudness

""" Time Signature (compás) """

def estimate_time_signature_value(y, sr):
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
    n_beats = len(beats)

    # Heurística simple para videojuegos
    # (solo interesa una aproximación numérica)
    if tempo < 70:
        return 3   # suave o vals
    elif tempo < 100:
        return 2   # binario, marchas
    else:
        return 4   # casi todo lo demás (loops, combate, etc.)

In [76]:
import pandas as pd
import os


audio_files = ["data/27. It’s TV Time!.mp3"] 
all_features = []

for audio_path in audio_files:
    # Cargar audios
    y, sr = librosa.load(audio_path, sr=None)
    audio_essentia = es.MonoLoader(filename=audio_path)()
    
    # Extraer todas las features
    tempo = get_tempo(y, sr)
    key, scale = get_key(audio_essentia)
    loudness = get_loudness(audio_essentia)
    time_signature = estimate_time_signature_value(y, sr)
    
    # Crear un diccionario para esta canción
    features_dict = {
        'filename': os.path.basename(audio_path),
        'tempo': tempo,
        'key': key,
        'scale': scale,
        'loudness': loudness,
        'time_signature': time_signature
    }
    
    all_features.append(features_dict)

# 2. Convertir la lista de diccionarios a un DataFrame
features_df = pd.DataFrame(all_features)

# 3. Guardar el DataFrame
print(features_df)

[148.02631579]
                filename                 tempo key  scale     loudness  \
0  27. It’s TV Time!.mp3  [148.02631578947367]   G  minor  4617.086426   

   time_signature  
0               4  
